# Finding optimal values for A1 and A2 coefficient

Given an "ideal" initial snow density, we try to determine what is the best pair of (A1, A2) coefficients, using 3 different metrics (see Notebook 19).

**WARNING: THE OPTIMIZATION MAY DEPEND ON THE OFFSET VALUE > PERHAPS A GRAPH MAY BE HELPFUL.**

In [1]:
# TODO re-write the top part (and title?)

## Imports and functions

In [2]:
from joblib import Parallel, delayed
import math
from scipy.stats import sem
from matplotlib.patches import Rectangle

In [3]:
%run little_awk_functions.py
%run parameters.py

In get_met_forcing() - Warning: check the format of your file corresponds to the indices given in the functions (wind speed 5, surface temperature 8, time 0).


In [4]:
def single_simulation(data_set_used, x_sel, y_sel, data_duration_in_s, ds_time_indices, end_accumulation_times, end_erosion_times, start_accumulation, end_accumulation, 
                      start_erosion, end_erosion, jj, dt, ro_layer, ro_water, ro_ice, t_old, tf, tsfc, dy_snow, age_layers, gamma, cp_snow, melt_flag, a1, a2,
                      met_temp, met_time, simul_new_snow_ro, simul_fit_top_of_snowfall_to_curve):
    '''
    Function that uses the simulate_snowpack_evolution() function to simulate the snowpack's evolution for a given (a1, a2) pair
    Args:
        data_duration_in_s: total duration of the lidar data, in seconds
        ds_time_indices: indices of the simulation that correspond to the lidar data (and should thus be the only to be kept)
        for all other arguments, please refer to the simulate_snowpack_evolution() docstring
    Returns:
        an np-array containing:
        a1, a2: unchanged values, used for identification of the results
        total_snow_depth: array containing the depth of the snowpack at each timestamp
    '''
    nb_iterations = int(data_duration_in_s/dt + 1)

    # Update variables at each timepoint

    snowpack = simulate_snowpack_evolution(data_set_used, x_sel, y_sel, nb_iterations, end_accumulation_times, end_erosion_times,
                                       start_accumulation, end_accumulation, start_erosion, end_erosion, jj, dt, ro_layer, ro_water, ro_ice,
                                       t_old, tf, tsfc, dy_snow, age_layers, gamma, cp_snow, melt_flag, a1, a2, met_temp_data=met_temp,
                                       met_time_data=met_time, new_snow_ro=simul_new_snow_ro,
                                       fit_top_of_snowfall_to_curve=simul_fit_top_of_snowfall_to_curve)

    ro_layer_evolution, depth_evolution, temperature_evolution = snowpack[0], snowpack[1], snowpack[2]
     
    # Define total_snow_depth
    
    # simulation_times = pd.date_range(start=data_start_date,freq=str(dt)+'S',periods=nb_iterations)
    total_snow_depth = np.array([sum(depth_evolution[i][j] for j in range(max_nb_of_layers)) for i in range(len(depth_evolution)) if i in ds_time_indices])
    
    return(np.array([a1, a2, total_snow_depth], dtype=object))

In [5]:
def parallel_simulation(data_set_used, x_sel, y_sel, data_duration_in_s, ds_time_indices, end_accumulation_times, end_erosion_times, start_accumulation, end_accumulation, 
                      start_erosion, end_erosion, jj, dt, ro_layer, ro_water, ro_ice, t_old, tf, tsfc, dy_snow, age_layers, gamma, cp_snow, melt_flag, a1_range, a2_range,
                      met_temp, met_time, simul_new_snow_ro, simul_fit_top_of_snowfall_to_curve):
    '''
    Function that runs single_simulation() in parallel for a range of values of a1 and a2
    Args:
        data_duration_in_s: total duration of the lidar data, in seconds
        ds_time_indices: indices of the simulation that correspond to the lidar data (and should thus be the only to be kept)
        for all other arguments, please refer to the simulate_snowpack_evolution() docstring
    Returns:
        p_s: array containing the [a1, a2, total_snow_depth] results computed for each pair of (a1, a2) arguments
    '''
    a1_a2_range = [(a1, a2) for a1 in a1_range for a2 in a2_range]
    
    p_s = Parallel(n_jobs=-2)(delayed(single_simulation)(data_set_used, x_sel, y_sel, data_duration_in_s, ds_time_indices, end_accumulation_times, end_erosion_times,
                      start_accumulation, end_accumulation, start_erosion, end_erosion, jj, dt, ro_layer, ro_water, ro_ice, t_old, tf, tsfc, dy_snow, age_layers, gamma, cp_snow,
                      melt_flag, a1, a2, met_temp, met_time, simul_new_snow_ro, simul_fit_top_of_snowfall_to_curve) for (a1, a2) in a1_a2_range)
    
    return(np.array(p_s, dtype=object))

In [6]:
# convert lidar timepoints to match simulation times > see first metrics functions

def parallel_measures(parallel_simulations_array, lidar_height_array):
    '''
    Function that runs all_measures() in parallel for a range of values of a1 and a2
    Args:
        parallel_simulations_array: array containing the [a1, a2, total_snow_depth] results of simulations computed for each pair of (a1, a2) arguments
        lidar_height_array: array containing the depth of the snowpack as measured by the lidar, corresponding to each simulation timestamp
    Returns:
        p_m: array containing the [a1, a2, (rmse, stde, p_corr)] measure results computed for each pair of (a1, a2) arguments
    '''
    p_m = Parallel(n_jobs=-2)(delayed(all_measures)(parallel_simulations_array[i][0], parallel_simulations_array[i][1], parallel_simulations_array[i][2], lidar_height_array)  # a1, a2, simul_total_height_array
                       for i in range(len(parallel_simulations_array)))
    
    return(p_m)

In [7]:
# TODO add all those functions to little_awk_functions?

## Define default parameters

In [8]:
# Dataset

name_of_data_set = 'snow_pit_1_filled.nc'

save_figures = False
save_text_results = False

directory_to_save_figs_in = '/home/mabonnet/github/MB_little_awk/current_development/A1_A2_optimum/'

In [9]:
# Events detection parameters

x_sel = 10
y_sel = 10

In [10]:
# Compaction/temperature model parameters

tsfc = -5
cp_snow = 2106
dt = 100
a1 = 0.0013
a2 = 0.021

use_true_temp = False   # set to True if want to use the correct temperature forcing

# TODO add ice layer detection option > not yet

# Will not be varied for now

max_nb_of_layers = 25
simul_fit_top_of_snowfall_to_curve = False
tf = 0
ro_water = 1000
ro_ice = 910
jj = 0

In [11]:
a1_range = np.linspace(0.001, 0.005, num=15, endpoint=True)
a2_range = np.linspace(0.02, 0.04, num=10, endpoint=True)

In [12]:
# Couples of values to plot the lidar graph of

couples_to_plot = [(a1_range[1], a2_range[1]), (a1_range[-2], a2_range[-2]), (a1_range[-2], a2_range[1]), (a1_range[1], a2_range[-2]), (a1_range[7], a2_range[8])]
print(couples_to_plot)

[(0.0012857142857142859, 0.022222222222222223), (0.004714285714285714, 0.03777777777777778), (0.004714285714285714, 0.022222222222222223), (0.0012857142857142859, 0.03777777777777778), (0.003, 0.03777777777777778)]


## Clean dataset and derive other parameters

In [13]:
# Pre-processing to get clean data

data_set_used = xr.open_dataset(name_of_data_set)

data_set_used = data_set_used.ffill(dim='time')

median_space_filtering(data_set_used, 5, x_span=7)
median_time_filtering(data_set_used, 11)

data_set_used['snow_surface'] = data_set_used['snow_surface'] - data_set_used['snow_surface'].isel(x=x_sel, y=y_sel).dropna('time').min()


---> Median filtering in space with a window [7, 11]
---> Median filtering in time with a window of 11


In [14]:
# Define dates

data_starting_date_in_ns = float(data_set_used.time.values[0])

data_starting_date_in_s = pd.to_datetime(data_set_used.time.values[0]).timestamp()
data_ending_date_in_s = pd.to_datetime(data_set_used.time.values[-1]).timestamp()
data_duration_in_s = data_ending_date_in_s - data_starting_date_in_s
nb_iterations = int(data_duration_in_s/dt + 1)

data_start_date = pd.to_datetime(data_set_used.time.values[0])

In [15]:
results = get_snow_events(data_set_used, x_sel, y_sel, time_window_std, std_threshold)
start_accumulation, start_erosion, end_accumulation, end_erosion = results[0], results[1], results[2], results[3]

# Convert end times into more manageable orders of magnitude

end_accumulation_times = data_set_used.snow_surface.isel(x=x_sel, y=y_sel, time=end_accumulation)
end_accumulation_times = (pd.to_datetime(end_accumulation_times.time).astype(int) - data_starting_date_in_ns) / 1000000000  # in s

end_erosion_times = data_set_used.snow_surface.isel(x=x_sel, y=y_sel, time=end_erosion)
end_erosion_times = (pd.to_datetime(end_erosion_times.time).astype(int) - data_starting_date_in_ns) / 1000000000  # in s

In [16]:
lidar_height_array = []
keep_simul_times_indices = []    # indices of the timestamps to keep in the simulation data, that are comparable to the lidar data
                                # given that dt << lidar scans time period, there are no repetitions in keep_simul_times_indices
ignore = np.isnan(data_set_used.snow_surface.isel(x=x_sel, y=y_sel))     # do not take into account the nan values in the dataset

for index in range(len(data_set_used.time.values)):
        lidar_time_in_s = float(data_set_used.time.values[index]) / 1000000000 - float(data_set_used.time.values[0]) / 1000000000
        if lidar_time_in_s < nb_iterations*dt and not ignore[index]:
            lidar_height_array.append(data_set_used.snow_surface.isel(x=x_sel, y=y_sel, time=index))

            index_of_closest_time_in_simul = int(lidar_time_in_s//dt + round((lidar_time_in_s%dt)/dt))
            keep_simul_times_indices.append(index_of_closest_time_in_simul)

lidar_height_array = np.array(lidar_height_array)

## Loop on parameters A1 and A2

In [17]:
simul = parallel_simulation(data_set_used, x_sel, y_sel, data_duration_in_s, keep_simul_times_indices, end_accumulation_times, end_erosion_times, start_accumulation, end_accumulation, 
                      start_erosion, end_erosion, jj, dt, ro_layer, ro_water, ro_ice, t_old, tf, tsfc, dy_snow, age_layers, gamma, cp_snow, melt_flag, a1_range, a2_range,
                      met_temp, met_time, simul_new_snow_ro, simul_fit_top_of_snowfall_to_curve)


NameError: name 'simul_new_snow_ro' is not defined

In [ ]:
measrs = parallel_measures(simul, lidar_height_array)


In [ ]:
for i in range(len(measrs)):
    if measrs[i][2][2] == max(measrs[i][2][2] for i in range(len(measrs))):
        print('Pearson correlation max: ', measrs[i])
    if measrs[i][2][1] == min(measrs[i][2][1] for i in range(len(measrs))):
        print('Std error min: ', measrs[i])
    if measrs[i][2][0] == min(measrs[i][2][0] for i in range(len(measrs))):
        print('RMSE min: ', measrs[i])


In [ ]:
# RMSE measure > min is best

x = a1_range
y = a2_range
z = np.array([measrs[i][2][0] for i in range(len(measrs))])

X, Y = np.meshgrid(x, y)
Z = z.reshape(len(x), len(y)).T

plt.pcolor(X, Y, Z)
plt.xlabel('a1')
plt.ylabel('a2')

plt.colorbar()
plt.show()

In [ ]:
# Std error measure > min is best

x = a1_range
y = a2_range
z = np.array([measrs[i][2][1] for i in range(len(measrs))])

X, Y = np.meshgrid(x, y)
Z = z.reshape(len(x), len(y)).T

plt.pcolor(X, Y, Z)
plt.xlabel('a1')
plt.ylabel('a2')

plt.colorbar()
plt.show()

In [ ]:
# Pearson correlation measure > max is best

x = a1_range
y = a2_range
z = np.array([measrs[i][2][2] for i in range(len(measrs))])

X, Y = np.meshgrid(x, y)
Z = z.reshape(len(x), len(y)).T

plt.pcolor(X, Y, Z)
plt.xlabel('a1')
plt.ylabel('a2')

plt.colorbar()
plt.show()

## Plot results

In [ ]:
a1_step = a1_range[1]-a1_range[0]
a2_step = a2_range[1]-a2_range[0]

optimum_coords = [5, 5]

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(7, 10))

for ax in axs.flat:
    ax.set(xlabel='a1', ylabel='a2')
    # ax.set_xticks(a1_range * 1000, ['%d' % val for val in a1_range])
    # ax.set_xticks(a1_range)# * 100) 
    # ax.set_xticklabels(['%d' % val for val in a1_range])
    ax.label_outer()

x = a1_range
y = a2_range
z_rmse = np.array([measrs[i][2][0] for i in range(len(measrs))])
z_corr = np.array([measrs[i][2][2] for i in range(len(measrs))])

X, Y = np.meshgrid(x, y)
Z_rmse = z_rmse.reshape(len(x), len(y)).T
Z_corr = z_corr.reshape(len(x), len(y)).T

fig0 = axs[0].pcolor(X, Y, Z_rmse)
axs[0].set_title('Root Mean Square Error')
fig.colorbar(fig0, ax=axs[0])

axs[0].add_patch(Rectangle((a1_range[1] - a1_step/2, a2_range[1] - a2_step/2), a1_step, a2_step, edgecolor='white', facecolor='none', lw=1, ls='-'))
axs[0].add_patch(Rectangle((a1_range[-2] - a1_step/2, a2_range[-2] - a2_step/2), a1_step, a2_step, edgecolor='white', facecolor='none', lw=1, ls='-'))
axs[0].add_patch(Rectangle((a1_range[-2] - a1_step/2, a2_range[1] - a2_step/2), a1_step, a2_step, edgecolor='white', facecolor='none', lw=1, ls='-'))
axs[0].add_patch(Rectangle((a1_range[1] - a1_step/2, a2_range[-2] - a2_step/2), a1_step, a2_step, edgecolor='white', facecolor='none', lw=1, ls='-'))
axs[0].add_patch(Rectangle((a1_range[optimum_coords[0]] - a1_step/2, a2_range[optimum_coords[1]] - a2_step/2), a1_step, a2_step, edgecolor='white', facecolor='none', lw=1, ls='-'))   # TODO last one should be modular

fig1 = axs[1].pcolor(X, Y, Z_corr)
axs[1].set_title('Pearson correlation')
fig.colorbar(fig1, ax=axs[1])

axs[1].add_patch(Rectangle((a1_range[1] - a1_step/2, a2_range[1] - a2_step/2), a1_step, a2_step, edgecolor='white', facecolor='none', lw=1, ls='-'))
axs[1].add_patch(Rectangle((a1_range[-2] - a1_step/2, a2_range[-2] - a2_step/2), a1_step, a2_step, edgecolor='white', facecolor='none', lw=1, ls='-'))
axs[1].add_patch(Rectangle((a1_range[-2] - a1_step/2, a2_range[1] - a2_step/2), a1_step, a2_step, edgecolor='white', facecolor='none', lw=1, ls='-'))
axs[1].add_patch(Rectangle((a1_range[1] - a1_step/2, a2_range[-2] - a2_step/2), a1_step, a2_step, edgecolor='white', facecolor='none', lw=1, ls='-'))
axs[1].add_patch(Rectangle((a1_range[optimum_coords[0]] - a1_step/2, a2_range[optimum_coords[1]] - a2_step/2), a1_step, a2_step, edgecolor='white', facecolor='none', lw=1, ls='-'))   # TODO last one should be modular


In [ ]:
optimum_index = optimum_coords[0]*len(a2_range) + optimum_coords[1]

fig, axs = plt.subplots(5, 1, figsize=(10, 10))

for ax in axs.flat:
    ax.set(xlabel='time', ylabel='snow height (m)')
    ax.label_outer()

axs[0].plot(simul[len(a2_range)+1][2], label='a1 = '+str(round(simul[len(a2_range)+1][0], 4))+', a2 = '+str(round(simul[len(a2_range)+1][1], 3)))
axs[0].plot(data_set_used.snow_surface.isel(x=x_sel, y=y_sel), label='lidar data')
axs[0].legend()

axs[1].plot(simul[-2*len(a2_range)+(len(a2_range)-2)][2], label='a1 = '+str(round(simul[-2*len(a2_range)+(len(a2_range)-2)][0], 4))+', a2 = '+str(round(simul[-2*len(a2_range)+(len(a2_range)-2)][1], 3)))
axs[1].plot(data_set_used.snow_surface.isel(x=x_sel, y=y_sel), label='lidar data')
axs[1].legend()

axs[2].plot(simul[len(a2_range)+(len(a2_range)-2)][2], label='a1 = '+str(round(simul[len(a2_range)+(len(a2_range)-2)][0], 4))+', a2 = '+str(round(simul[len(a2_range)+(len(a2_range)-2)][1], 3)))
axs[2].plot(data_set_used.snow_surface.isel(x=x_sel, y=y_sel), label='lidar data')
axs[2].legend()

axs[3].plot(simul[-2*len(a2_range)+1][2], label='a1 = '+str(round(simul[-2*len(a2_range)+1][0], 4))+', a2 = '+str(round(simul[-2*len(a2_range)+1][1], 3)))
axs[3].plot(data_set_used.snow_surface.isel(x=x_sel, y=y_sel), label='lidar data')
axs[3].legend()

axs[4].plot(simul[optimum_index][2], label='a1 = '+str(round(simul[optimum_index][0], 4))+', a2 = '+str(round(simul[optimum_index][1], 3)))
axs[4].plot(data_set_used.snow_surface.isel(x=x_sel, y=y_sel), label='lidar data')
axs[4].legend()

fig.suptitle('Simulated vs measured snow height for different values of a1, a2')
fig.tight_layout()
fig.subplots_adjust(top=0.93)

In [ ]:
# Other option for presentation of results (change fig sizes, make first one horizontal for eg.)

optimum_index = optimum_coords[0]*len(a2_range) + optimum_coords[1]

fig = plt.figure(figsize=(15, 5))

fig.xlabel='time'
fig.ylabel='snow height (m)'

plt.plot(simul[len(a2_range)+1][2], label='a1 = '+str(round(simul[len(a2_range)+1][0], 4))+', a2 = '+str(round(simul[len(a2_range)+1][1], 3)))
plt.plot(simul[-2*len(a2_range)+(len(a2_range)-2)][2], label='a1 = '+str(round(simul[-2*len(a2_range)+(len(a2_range)-2)][0], 4))+', a2 = '+str(round(simul[-2*len(a2_range)+(len(a2_range)-2)][1], 3)))
plt.plot(simul[len(a2_range)+(len(a2_range)-2)][2], label='a1 = '+str(round(simul[len(a2_range)+(len(a2_range)-2)][0], 4))+', a2 = '+str(round(simul[len(a2_range)+(len(a2_range)-2)][1], 3)))
plt.plot(simul[-2*len(a2_range)+1][2], label='a1 = '+str(round(simul[-2*len(a2_range)+1][0], 4))+', a2 = '+str(round(simul[-2*len(a2_range)+1][1], 3)))

plt.plot(simul[optimum_index][2], label='a1 = '+str(round(simul[optimum_index][0], 4))+', a2 = '+str(round(simul[optimum_index][1], 3)))

plt.plot(data_set_used.snow_surface.isel(x=x_sel, y=y_sel), label='lidar data')
plt.legend()

plt.title('Simulated vs measured snow height for different values of a1, a2')

# TODO adjust colors and alphas

In [ ]:
# TODO: find a way of identifying each couple-point to the corresponding graph > put numbers